# Test On-The-Fly Photometry for Daily Reductions

In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column
from desiutil.names import radec_to_desiname
from desiutil.log import get_logger
from desispec.io.photo import gather_tractorphot, gather_targetphot
from desitarget.targets import decode_targetid
import specprodDB.load as db
specprod = os.environ['SPECPROD']
tileid = 3867
survey, program = 'main', 'dark'

## Database connection

In [ ]:
db.log = get_logger()
postgresql = db.setup_db(schema='iron', hostname='specprod-db.desi.lbl.gov', username='desi')

In [ ]:
tile = db.dbSession.query(db.Tile).filter(db.Tile.tileid == tileid).one()
tile

In [ ]:
assert tile.survey == survey
assert tile.program == program

In [ ]:
len(tile.targets)

In [ ]:
def no_sky(catalog):
    """Identify objects in `catalog` that are not sky targets.
    
    Parameters
    ----------
    catalog : :class:`~astropy.table.Table`
        Any Table containing a ``TARGETID`` column.
    
    Returns
    -------
    :class:`numpy.ndarray`
        The indexes of rows that are not sky targets.
    """
    _, _, _, _, sky, _ = decode_targetid(catalog['TARGETID'])
    return np.where((sky == 0) & (catalog['TARGETID'] > 0))[0]

## Load a random tile

In [ ]:
fiberassign = os.path.join(os.environ['DESI_ROOT'], 'target', 'fiberassign', 'tiles', 'trunk', f'{tileid:06d}'[0:3], f'fiberassign-{tileid:06d}.fits.gz')
with fits.open(fiberassign) as hdulist:
    hdulist.info()
    observed_targets = hdulist['FIBERASSIGN'].data
    potential_targets = hdulist['TARGETS'].data

no_sky_targets = no_sky(observed_targets)
potential_no_sky_targets = no_sky(potential_targets)
observed_cat = Table()
observed_cat['TARGETID'] = observed_targets['TARGETID'][no_sky_targets]
observed_cat['TILEID'] = tileid
observed_cat['TARGET_RA'] = observed_targets['TARGET_RA'][no_sky_targets]
observed_cat['TARGET_DEC'] = observed_targets['TARGET_DEC'][no_sky_targets]
observed_cat['PETAL_LOC'] = observed_targets['PETAL_LOC'][no_sky_targets]
observed_cat['SURVEY'] = survey
observed_cat['PROGRAM'] = program
potential_cat = Table()
potential_cat['TARGETID'] = potential_targets['TARGETID'][potential_no_sky_targets]
potential_cat['TILEID'] = tileid
potential_cat['TARGET_RA'] = potential_targets['RA'][potential_no_sky_targets]
potential_cat['TARGET_DEC'] = potential_targets['DEC'][potential_no_sky_targets]
# potential_cat['PETAL_LOC'] = potential_targets['PETAL_LOC'][potential_no_sky_targets]
potential_cat['SURVEY'] = survey
potential_cat['PROGRAM'] = program

In [ ]:
observed_cat

In [ ]:
potential_cat

## Tractor photometry for observed targets

In [ ]:
tractorphot = gather_tractorphot(observed_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [ ]:
tractorphot

## Target information for observed targets

In [ ]:
targetphot = gather_targetphot(observed_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
targetphot['SURVEY'] = observed_cat['SURVEY']
targetphot['PROGRAM'] = observed_cat['PROGRAM']
targetphot['TILEID'] = observed_cat['TILEID']
targetphot['DESINAME'] = radec_to_desiname(targetphot['RA'], targetphot['DEC'])
inan = np.logical_or(np.isnan(targetphot['PMRA']), np.isnan(targetphot['PMDEC']))
if np.any(inan):
    targetphot['PMRA'][inan] = 0.0
    targetphot['PMDEC'][inan] = 0.0
targetphot

## Tractor photometry for potential targets

In [ ]:
potential_tractorphot = gather_tractorphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')

In [ ]:
potential_tractorphot

## Target information for potential targets

In [ ]:
potential_targetphot = gather_targetphot(potential_cat, racolumn='TARGET_RA', deccolumn='TARGET_DEC')
potential_targetphot['SURVEY'] = potential_cat['SURVEY']
potential_targetphot['PROGRAM'] = potential_cat['PROGRAM']
potential_targetphot['TILEID'] = potential_cat['TILEID']
potential_targetphot['DESINAME'] = radec_to_desiname(potential_targetphot['RA'], potential_targetphot['DEC'])
inan = np.logical_or(np.isnan(potential_targetphot['PMRA']), np.isnan(potential_targetphot['PMDEC']))
if np.any(inan):
    potential_targetphot['PMRA'][inan] = 0.0
    potential_targetphot['PMDEC'][inan] = 0.0
potential_targetphot

## Compare "gather" results to the database

In [ ]:
assert len(potential_targetphot) == len(tile.targets)

In [ ]:
for t in tile.targets:
    p = t.photometry
    i = np.where(potential_targetphot['TARGETID'] == t.targetid)[0]
    j = np.where(potential_tractorphot['TARGETID'] == t.targetid)[0]
    row = potential_targetphot[i]
    phot_row = potential_tractorphot[i]
    assert t.tileid == tileid
    assert t.survey == survey
    assert t.program == program
    for col in ('DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'SCND_TARGET'):
        assert row[col] == getattr(t, col.lower())
    for col in ('BRICKID', 'BRICKNAME', 'TYPE', 'MASKBITS'):
        if col == 'TYPE':
            assert phot_row[col] == getattr(p, 'morphtype')
        else:
            assert phot_row[col] == getattr(p, col.lower())